## Imports

In [ ]:
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Google Drive connection
For this project, I will store the data on my personal google drive and below set up a connection to it. So, I can load the CSV files as they are from a system folder.

In [ ]:
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
data_folder =  '/gdrive/My Drive/Data Science Career Track/Unit 7/capstone_two_project_folder/data/'

# Data Colleciton


## Data Loading

In [ ]:
file_assessments  = data_folder + 'assessments.csv'
file_courses = data_folder + 'courses.csv'
file_student_assessment = data_folder + 'studentAssessment.csv'
file_student_info = data_folder + 'studentInfo.csv'
file_student_register = data_folder + 'studentRegistration.csv'
file_student_vle = data_folder + 'studentVle.csv'
file_vle = data_folder + 'vle.csv'

In [ ]:
raw_assessments = pd.read_csv(file_assessments)
raw_courses = pd.read_csv(file_courses)
raw_student_assessment = pd.read_csv(file_student_assessment)
raw_student_info = pd.read_csv(file_student_info)
raw_student_register = pd.read_csv(file_student_register)
raw_student_vle = pd.read_csv(file_student_vle)
raw_vle = pd.read_csv(file_vle)

In [ ]:
raw_assessments.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0


In [ ]:
raw_courses.head()

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


In [ ]:
raw_student_assessment.head()

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0


In [ ]:
raw_student_info.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass


In [ ]:
raw_student_register.head()

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN


In [ ]:
raw_student_vle.head()

,code_module,code_presentation,id_student,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1


There is this one issue here. On the documentation, it says that `"sum_click – the number of times a student interacts with the material in that day"`. However, the first three rows are identical to the VLE material and the student but have different `sum_click` values.


In [ ]:
raw_vle.head()

,id_site,code_module,code_presentation,activity_type,week_from,week_to
0,546943,AAA,2013J,resource,NaN,NaN
1,546712,AAA,2013J,oucontent,NaN,NaN
2,546998,AAA,2013J,resource,NaN,NaN
3,546888,AAA,2013J,url,NaN,NaN
4,547035,AAA,2013J,resource,NaN,NaN


## Data Explore

### `student_vle`

First, I will have a look at the issue about `sum_click` and try to understand the issue and group the data by columns `'code_module', 'code_presentation', 'id_student', 'id_site', 'date'`

In [ ]:
grouped_student_vle = raw_student_vle.groupby(
    by=['code_module', 'code_presentation', 'id_student', 'id_site', 'date']
    ).sum('sum_click').reset_index()

In [ ]:
num_rows_raw = raw_student_vle.shape[0] 
num_rows_grouped = grouped_student_vle.shape[0]
num_rows_diff = num_rows_raw - num_rows_grouped
(num_rows_raw, num_rows_grouped, num_rows_diff, (num_rows_diff) * 100 / num_rows_raw)

(10655280, 8459320, 2195960, 20.609125241194974)

The total num of rows in the raw table is 10'655'280  
If the data is groupped by all the columns except `sum_clicks` this is reduced to 8'459'320  
This means a **%20.6** shrinkage and the difference is 2'195'960  

---  

At this point I will assume that the actual total num of clicks is the groupped one. (Actually got reply from Martin Hols -one of data providers- and he suggested the same way))

In [ ]:
# Duplicate entries
student_vle_duplicate = raw_student_vle.merge(
    grouped_student_vle,
    indicator = True,
    how='left',
    ).loc[lambda x : x['_merge']!='both']

Cross-check of the group by function, if the summation is correct.  
I will randomly select some course (means `code_module`, `code_presentation`),  student, site and date from `student_vle_duplicate`, then find the entry in both raw and grouped tables and compare. So I expect to find more than one row for each student on the `raw_student_vle` table and sum of the `sum_clicks` will be equal to the `sum_clicks` on the `grouped_student_vle`

In [ ]:
rand_idx = np.random.choice(student_vle_duplicate.index)
rand_row = student_vle_duplicate.loc[rand_idx]

multiple = raw_student_vle[
                (raw_student_vle['code_module'] == rand_row['code_module']) &
                (raw_student_vle['code_presentation'] == rand_row['code_presentation']) &
                (raw_student_vle['id_student'] == rand_row['id_student']) &
                (raw_student_vle['id_site'] == rand_row['id_site']) &
                (raw_student_vle['date'] == rand_row['date']) 
               ]

single = grouped_student_vle[
                (grouped_student_vle['code_module'] == rand_row['code_module']) &
                (grouped_student_vle['code_presentation'] == rand_row['code_presentation']) &
                (grouped_student_vle['id_student'] == rand_row['id_student']) &
                (grouped_student_vle['id_site'] == rand_row['id_site']) &
                (grouped_student_vle['date'] == rand_row['date']) 
               ]

multiple, single

(        code_module code_presentation  id_student  id_site  date  sum_click
 6047564         EEE             2014J      684064   832567   100          3
 6047565         EEE             2014J      684064   832567   100          2,
         code_module code_presentation  id_student  id_site  date  sum_click
 4849797         EEE             2014J      684064   832567   100          5)

Sum process looks successful and from now on `grouped_student_vle` will be used instead of `raw_student_vle`

### `student_info`

How to handle the `num_of_prev_attemts` column?   

In [ ]:
raw_student_info['num_of_prev_attempts'].unique()

array([0, 1, 2, 4, 3, 5, 6])

So there are some students with up to 6th trial.
    
Does their `id_student`change by trial?


In [ ]:
raw_student_info['id_student'].unique().shape[0], raw_student_info.shape[0]

(28785, 32593)

It doesn't. So how should these former attempts be handled? As an additional effect on final trial's result or as an individual trial without any connection?  

First o all, data is prepared for the years 2013 and 2014. Are there any students who is taking any course second (or more) time in the first semester of 2013 

In [ ]:
raw_student_info[
                 (raw_student_info['num_of_prev_attempts'] > 1) & 
                 (raw_student_info['code_presentation'] == '2013B')
                 ]

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
748,BBB,2013B,23629,F,East Anglian Region,Lower Than A Level,20-30%,0-35,2,60,N,Fail
750,BBB,2013B,27891,M,Scotland,Lower Than A Level,0-10%,0-35,2,120,Y,Withdrawn
754,BBB,2013B,34229,F,Ireland,A Level or Equivalent,0-10%,35-55,2,120,N,Pass
764,BBB,2013B,58089,F,West Midlands Region,A Level or Equivalent,40-50%,0-35,2,90,N,Fail
773,BBB,2013B,79378,F,London Region,Lower Than A Level,20-30%,35-55,4,120,Y,Fail
...,...,...,...,...,...,...,...,...,...,...,...,...
23706,FFF,2013B,962951,M,South Region,A Level or Equivalent,50-60%,35-55,2,150,N,Withdrawn
23760,FFF,2013B,1855249,F,South Region,HE Qualification,90-100%,35-55,2,60,N,Withdrawn
23797,FFF,2013B,2108821,M,Scotland,HE Qualification,90-100%,35-55,2,120,N,Pass
23832,FFF,2013B,2381745,M,London Region,Lower Than A Level,0-10%,35-55,2,150,N,Withdrawn


There are such rows, so we don't have any information about their former attempt. Therefore, all attempts will be handled on their own.

# Data Joining

Eventually, all the tables will be joined to `student_info`. But `assessment` and `vle` tables will be preprocessed and joined to `student_assessment` and `student_vle` tables respectively.



### Preparing `student_assesseent`
These tables will bring up information about assessments, like if it's tutor marked assignmet (`assessment_type == 'TMA'`) or the student's score on the assignment.

In [ ]:
student_assessment = (
    raw_student_assessment.set_index('id_assessment')
    .join(raw_assessments.set_index('id_assessment'))
).set_index(['code_module', 'code_presentation', 'id_student'])

At this point there are multiple entries for a `code_module`, `code_presentation`, `id_student`. So this table can not be added to `student_info`. I need to come up with some aggregation for student assessment

In [ ]:
# TODO: Find some aggregations

### Preparing `student_vle`
Thse table will bring up student activity on virtual learning platform. Such as, `activity_type` or `sum_click`

In [ ]:
student_vle = (
    grouped_student_vle.set_index(['code_module', 'code_presentation', 'id_site'])
    .join(raw_vle.set_index(['code_module', 'code_presentation', 'id_site']))
).reset_index()

In [ ]:
student_vle.head()

id_site  date  ...  week_from week_to
code_module code_presentation id_student                 ...                   
AAA         2013J             11391        546614    -5  ...        NaN     NaN
                              11391        546614     0  ...        NaN     NaN
                              11391        546614     1  ...        NaN     NaN
                              11391        546614     2  ...        NaN     NaN
                              11391        546614     6  ...        NaN     NaN

[5 rows x 6 columns]

At this point there are multiple entries for a `code_module`, `code_presentation`, `id_student`. So this table can not be added to `student_info`. I need to come up with some aggregation for student activity

In [ ]:
## TODO: Find out aggregate student activity on `student_vle` 

### Preparing `courses` 
This table will bring up `module_presentation_length` informaiton

In [ ]:
courses = raw_courses.set_index(['code_module', 'code_presentation'])

### Preparing `student_register`

In [ ]:
student_register = raw_student_register.set_index(['code_module', 'code_presentation', 'id_student'])

### Final Join 

In [ ]:
attempt = (
    raw_student_info
    .set_index(['code_module', 'code_presentation', 'id_student'])
    .join(courses)
    .join(student_register)

) 

In [ ]:
attempt.head()

gender  ... date_unregistration
code_module code_presentation id_student         ...                    
AAA         2013J             11391           M  ...                 NaN
                              28400           F  ...                 NaN
                              30268           F  ...                12.0
                              31604           F  ...                 NaN
                              32885           F  ...                 NaN

[5 rows x 12 columns]

# Data Definition
* Column names
* Data types
* Description of the columns
* Counts and percents unique values
* Ranges of values

# Data Cleaning
* NA or missing values
* Duplicates

 `student_info`

How a previous attempt should be considered?  Is it someting students build up their understanding in a long time? Or just another individual attempt?
 
Can student's former attempts'  can be added to their final trial. In other words, does student's learn something from failed courses and use it on their next trial?